In [ ]:
import os
from os.path import abspath, dirname, basename, join as pjoin, exists as pexists
from timeit import default_timer as timer
import psutil
import numpy as np
import pydicom
import lzma
from pymedphys.gamma._level2.gammashell import gamma_shell, gamma_loop, calculate_min_dose_difference, interpolate_evaluation_dose_at_distance, multi_thresholds_gamma_calc, add_shells_to_eval_coords
from pymedphys.dicom import coords_and_dose_from_dicom
from scipy.interpolate import RegularGridInterpolator

%load_ext line_profiler

In [ ]:
mem = psutil.virtual_memory()
print(mem.total)
print(mem.available)
print(mem.percent)
print("Available RAM: {:.1f}%".format(100*mem.available/mem.total))

In [ ]:
DATA_DIR = pjoin(dirname(dirname(os.getcwd())), "tests", "data")

dicom_dir = r'E:\Workspace\DICOM test files\6283'

path_eval = pjoin(dicom_dir, "RD.6283_Pin.dcm")
path_ref = pjoin(dicom_dir, "RD.6283_Ecl.dcm")

coords_eval, dose_eval = coords_and_dose_from_dicom(path_eval)
coords_ref, dose_ref = coords_and_dose_from_dicom(path_ref)

dose_ref_valid = np.ma.masked_where(dose_ref <= 0.1*np.max(dose_ref), dose_ref)

print("Number of dose points: {}".format(dose_ref.size))
print("Number of dose points above 10% threshold: {}".format(dose_ref_valid[~dose_ref_valid.mask].size))

In [ ]:
count=5000
def run_gamma(random_point_count=count):
    # Gamma shell, 10,000 random subset, max gamma = 1
    gamma = gamma_shell(coords_ref,
                        dose_ref,
                        coords_eval,
                        dose_eval,
                        dose_percent_threshold=3,
                        distance_mm_threshold=3,
                        lower_percent_dose_cutoff=10,
                        random_subset=random_point_count,
                        ram_available=2000000000                    
    )
    print("\nGamma with 10,000 random points and max gamma of 1 took {:.1f}s".format(end-start))
    print("Pass rate = {:.3f}%\n".format(calculate_pass_rate(gamma)))

In [ ]:

%lprun -f gamma_loop -f calculate_min_dose_difference -f interpolate_evaluation_dose_at_distance -f add_shells_to_eval_coords gamma_shell(coords_ref, dose_ref,coords_eval,dose_eval,dose_percent_threshold=3,distance_mm_threshold=3,lower_percent_dose_cutoff=10,max_gamma = 1.1,random_subset=count,ram_available=2000000000)